# DATA EXPLORATION GAS SALES

In [57]:
import os
import pandas as pd
from typing import List

In [58]:
# Define sample path
# DEMO_DATA_FILE_PATH = r"C:\Users\ecastaneda\Documents\Dataton\gas-sales-prediction\data\raw\2019\Ene 2019 ATQ Y EC.csv"
route_folder = os.path.join(os.getcwd(), 'src', 'data')
route_folder_todogas_raw = os.path.join(route_folder, 'TODOGAS')
DATA_FILE_PATH_BY_YEAR = os.path.join(route_folder_todogas_raw, '2019', 'ATQ Y EC')
FILES = os.listdir(DATA_FILE_PATH_BY_YEAR)
DEMO_DATA_FILE_PATH = os.path.join(DATA_FILE_PATH_BY_YEAR, FILES[0])
print(DEMO_DATA_FILE_PATH)

/mnt/c/Users/arios/Documents/repositories/gas-sales-prediction/notebooks/src/data/TODOGAS/2019/ATQ Y EC/Abr 2019 ATQ Y EC.csv


In [59]:
# Read df
sample_df = pd.read_csv(DEMO_DATA_FILE_PATH, low_memory=False, thousands=',')
sample_df.head(10)

,Empresa,Region,Filial,Tipo,Punto de venta,%Key_Cliente,%Key_FechaID,%Key_Folio,%Key_Ticket,Litros,$
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2217649,19879205
1,"MERCANTIL DISTRIBUIDORA, S.A. DE C.V.",Este,ALAMO,Autotanque,ATQ. ALA 05,A01463,20190404.0,3000-5268-4363-2000-54D6,yxF0SM,43,396
2,"MERCANTIL DISTRIBUIDORA, S.A. DE C.V.",Este,ALAMO,Autotanque,ATQ. ALA 05,A01463,20190410.0,3000-5268-A414-1800-555C,yxFJCF,8,78
3,"MERCANTIL DISTRIBUIDORA, S.A. DE C.V.",Este,ALAMO,Autotanque,ATQ. ALA 05,A01463,20190410.0,3000-5268-A418-C800-555D,yxFJCG,6,51
4,"MERCANTIL DISTRIBUIDORA, S.A. DE C.V.",Este,ALAMO,Autotanque,ATQ. ALA 05,A01463,20190411.0,3000-5268-B506-D800-5574,yxFNUJ,72,661
5,"MERCANTIL DISTRIBUIDORA, S.A. DE C.V.",Este,ALAMO,Autotanque,ATQ. ALA 05,A01463,20190413.0,3000-5268-D4AA-6800-55A4,yxFTHO,17,158
6,"MERCANTIL DISTRIBUIDORA, S.A. DE C.V.",Este,ALAMO,Autotanque,ATQ. ALA 05,A01463,20190413.0,3000-5268-D70D-C800-55B2,yxFTIP,29,263
7,"MERCANTIL DISTRIBUIDORA, S.A. DE C.V.",Este,ALAMO,Autotanque,ATQ. ALA 05,A01463,20190413.0,3000-5268-D714-D000-55B3,yxFTIT,72,661
8,"MERCANTIL DISTRIBUIDORA, S.A. DE C.V.",Este,ALAMO,Autotanque,ATQ. ALA 05,A01463,20190414.0,3000-5268-E46E-C000-55BA,yxFVJV,20,185
9,"MERCANTIL DISTRIBUIDORA, S.A. DE C.V.",Este,ALAMO,Autotanque,ATQ. ALA 05,A01463,20190415.0,3000-5268-F50D-1800-55D2,yxFZPQ,8,78


In [60]:
# Show info
print(sample_df.shape)
sample_df.info()

(314897, 11)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 314897 entries, 0 to 314896
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Empresa         314896 non-null  object 
 1   Region          314896 non-null  object 
 2   Filial          314896 non-null  object 
 3   Tipo            314896 non-null  object 
 4   Punto de venta  314896 non-null  object 
 5   %Key_Cliente    314896 non-null  object 
 6   %Key_FechaID    314896 non-null  float64
 7   %Key_Folio      314896 non-null  object 
 8   %Key_Ticket     314896 non-null  object 
 9   Litros          314897 non-null  int64  
 10  $               314897 non-null  int64  
dtypes: float64(1), int64(2), object(8)
memory usage: 26.4+ MB


In [61]:
# Remove totals
def remove_totals(data_df: pd.DataFrame, column: str) -> pd.DataFrame:
    filtered_df = data_df[data_df[column].notna()]
    return filtered_df

sample_df = remove_totals(sample_df,"Filial")
sample_df.shape

(314896, 11)

In [62]:
# Convert type
sample_df.convert_dtypes()

,Empresa,Region,Filial,Tipo,Punto de venta,%Key_Cliente,%Key_FechaID,%Key_Folio,%Key_Ticket,Litros,$
1,"MERCANTIL DISTRIBUIDORA, S.A. DE C.V.",Este,ALAMO,Autotanque,ATQ. ALA 05,A01463,20190404,3000-5268-4363-2000-54D6,yxF0SM,43,396
2,"MERCANTIL DISTRIBUIDORA, S.A. DE C.V.",Este,ALAMO,Autotanque,ATQ. ALA 05,A01463,20190410,3000-5268-A414-1800-555C,yxFJCF,8,78
3,"MERCANTIL DISTRIBUIDORA, S.A. DE C.V.",Este,ALAMO,Autotanque,ATQ. ALA 05,A01463,20190410,3000-5268-A418-C800-555D,yxFJCG,6,51
4,"MERCANTIL DISTRIBUIDORA, S.A. DE C.V.",Este,ALAMO,Autotanque,ATQ. ALA 05,A01463,20190411,3000-5268-B506-D800-5574,yxFNUJ,72,661
5,"MERCANTIL DISTRIBUIDORA, S.A. DE C.V.",Este,ALAMO,Autotanque,ATQ. ALA 05,A01463,20190413,3000-5268-D4AA-6800-55A4,yxFTHO,17,158
...,...,...,...,...,...,...,...,...,...,...,...
314892,"MERCANTIL DISTRIBUIDORA, S.A. DE C.V.",Sur,TAMPICO,Autotanque,ATQ. TAMP 19 (029),TC0002,20190404,3001-3268-45DA-3800-2E27,yykB6Z,46,422
314893,"MERCANTIL DISTRIBUIDORA, S.A. DE C.V.",Sur,TAMPICO,Autotanque,ATQ. TAMP 19 (029),TC0002,20190404,3001-3268-45F6-5000-2E28,yykB71,6,53
314894,"MERCANTIL DISTRIBUIDORA, S.A. DE C.V.",Sur,TAMPICO,Autotanque,ATQ. TAMP 19 (029),TC0002,20190404,3001-3268-4614-9000-2E29,yykB72,9,80
314895,"MERCANTIL DISTRIBUIDORA, S.A. DE C.V.",Sur,TAMPICO,Autotanque,ATQ. TAMP 19 (029),TC0002,20190404,3001-3268-46C1-D000-2E2A,yykB74,15,133


In [63]:
# Get date
def keydate_to_date(data_df: pd.DataFrame, column: str = "%Key_FechaID") -> pd.Series:
    date_series = pd.to_datetime(data_df[column].astype(str), format = "%Y%m%d")
    return date_series

sample_df["Date"] = keydate_to_date(sample_df, "%Key_FechaID")

In [64]:
sample_df["Filial"].unique()

array(['ALAMO', 'ALDAMA', 'ALLENDE', 'CD. VICTORIA', 'GALEANA', 'JIMENEZ',
       'LINARES', 'MATEHUALA', 'MONTEMORELOS', 'POZA RICA', 'REYNOSA',
       'RIO VERDE', 'SALINASVICTORIA', 'SALTILLO', 'SAN FERNANDO',
       'SAN LUIS POTOSI', 'SAN NICOLAS', 'SAN RAFAEL', 'SANTA CATARINA',
       'SOTO LA MARINA', 'TAMPICO'], dtype=object)

In [65]:
# Remove columns
COLUMNS_TO_REMOVE = ["Empresa", "%Key_Ticket","%Key_Folio", "%Key_Cliente", "%Key_FechaID"]
sample_df.drop(columns = COLUMNS_TO_REMOVE, inplace=True)
sample_df.sample(10)

,Region,Filial,Tipo,Punto de venta,Litros,$,Date
70333,Oeste,MATEHUALA,Autotanque,ATQ. MATEH 96,1,14,2019-04-09
257508,Oeste,SAN RAFAEL,Autotanque,ATQ. RAI-54,2,21,2019-04-23
149171,Centro,REYNOSA,Estacion de Carburacion,EC SAN VALENTIN,5,48,2019-04-28
202177,Centro,SAN NICOLAS,Autotanque,ATQ. MTY 31,1,10,2019-04-03
147796,Centro,REYNOSA,Estacion de Carburacion,EC VILLA ESMERALDA,4,36,2019-04-29
19857,Este,CD. VICTORIA,Autotanque,ATQ. VIC 58 (438),3,27,2019-04-04
80232,Oeste,MATEHUALA,Autotanque,ATQ. MATEH 87,1,11,2019-04-26
299396,Sur,TAMPICO,Autotanque,ATQ. TAMP 87,3,24,2019-04-17
85945,Este,MONTEMORELOS,Autotanque,ATQ. MOR 22,2,22,2019-04-06
230587,Centro,SAN NICOLAS,Estacion de Carburacion,E.C. TERMOROMA,3,24,2019-04-23


In [66]:
file_filters = os.path.join(route_folder, 'FILTERS', 'filiales_estados.csv')
filters_df = pd.read_csv(file_filters, low_memory=False, thousands=',')
new_df = pd.merge(filters_df, sample_df, on="Filial")

In [67]:
YEARS: list = [2019, 2020, 2021]
COLUMNS_TO_REMOVE = ["Empresa", "%Key_Ticket","%Key_Folio", "%Key_Cliente", "%Key_FechaID"]

def extract_files_by_year(path_base: str, year: str):
    PATH_YEAR = os.path.join(path_base, str(year), 'EXPERIMENT')
    if not os.path.exists(PATH_YEAR):
        os.makedirs(PATH_YEAR)
    DATA_FILE_PATH_BY_YEAR = os.path.join(path_base, str(year))
    PATH_FILES = os.listdir(DATA_FILE_PATH_BY_YEAR)
    for file in PATH_FILES:
        FILE_PATH = os.path.join(DATA_FILE_PATH_BY_YEAR, file)
        s_df = pd.read_csv(FILE_PATH, low_memory=False, thousands=',')
        s_df = remove_totals(s_df, "Filial")
        s_df.convert_dtypes()
        s_df["Date"] = keydate_to_date(s_df, "%Key_FechaID")
        s_df.drop(columns = COLUMNS_TO_REMOVE, inplace=True)
        filters_df = pd.read_csv(file_filters, low_memory=False, thousands=',')
        s_df = pd.merge(filters_df, s_df, on="Filial")
        s_df.to_csv(f'{PATH_YEAR}/{file}')

In [68]:
def merge_files(path_base, year):
    # 1. creates empty list to include the content of each file converted to pandas DF
    csv_list = []
    PATH_YEAR = os.path.join(path_base, str(year), 'EXPERIMENT')
    PATH_FILES = os.listdir(PATH_YEAR)
    for file in sorted(PATH_FILES):
        FILE_PATH = os.path.join(PATH_YEAR, file)
        csv_list.append(pd.read_csv(FILE_PATH, low_memory=False, thousands=','))
    
    # 5. merges single pandas DFs into a single DF, index is refreshed 
    csv_merged = pd.concat(csv_list, ignore_index=True)

    file_all_name = f'{PATH_YEAR}/todo_gas_full_{year}.csv'

    if(os.path.exists(file_all_name) and os.path.isfile(file_all_name)):
        os.remove(file_all_name)

    # 6. Single DF is saved to the path in CSV format, without index column
    csv_merged.to_csv(file_all_name, index=False)
    

In [69]:
for year in YEARS:
    extract_files_by_year(route_folder_todogas_raw, year)

In [70]:
for year in YEARS:
    merge_files(route_folder_todogas_raw, year)